Drugi domaci iz masinskog ucenja.
Milos Stanisavljevic
Mladen Jovanovic
Miljana Stanic

# Za 1 zadatak ostaviti kolone:
Na osnovu specifikacije projekta i date tabele, zakljucili smo da treba ostaviti sledece tabele:

> Development data


**W_dev**

**X_s_dev**

**Y_dev**

**A_dev**

> Test data


**W_test**

**X_s_test**

**Y_test**

**A_test**

> Variables Name


**W_var**

**X_s_var**

**A_var**

# Za 1 zadatak obrisati kolone:

###   *Measurments, xs*
Na osnovu datih plot grafova zakljucili smo da P21 i P15 imaju isti plot grafik odnosno iste podatke o koloni, odakle smo zakljucili da ostavimo samo jednu, i ostavili smo P21.

  > Obrisati: **P15**


### *Pomoćni podaci (Auxiliary data):*

  > Obrisati: **Flight Class**

### *Health measurments:*

  > Obrisati:

   **fan_eff_mod**

   **fan_flow_mod**

   **LPC_eff_mod**

   **LPC_flow_mod** 

   **HPC_eff_mod** 

   **HPC_flow_mod**



In [ ]:
#Domaci 2 masinsko ucenje
import os
import h5py
import time
import matplotlib
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
from pandas import DataFrame
import matplotlib.pyplot as plt
from matplotlib import gridspec
import math
from functools import reduce 
%matplotlib inline


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
filename = '/content/drive/MyDrive/MasinskoD2/N-CMAPSS_DS02-006.h5'

Mounted at /content/drive


In [ ]:
# Load data
with h5py.File(filename, 'r') as hdf:
    # Development set
    W_dev = np.array(hdf.get('W_dev'))             # W
    X_s_dev = np.array(hdf.get('X_s_dev'))         # X_s
    Y_dev = np.array(hdf.get('Y_dev'))             # RUL  
    A_dev = np.array(hdf.get('A_dev'))             # Auxiliary

    # Test set
    W_test = np.array(hdf.get('W_test'))           # W
    X_s_test = np.array(hdf.get('X_s_test'))       # X_s
    Y_test = np.array(hdf.get('Y_test'))           # RUL  
    A_test = np.array(hdf.get('A_test'))           # Auxiliary

    # Varnams
    W_var = np.array(hdf.get('W_var'))
    X_s_var = np.array(hdf.get('X_s_var'))  
    A_var = np.array(hdf.get('A_var'))

    # from np.array to list dtype U4/U5
    W_var = list(np.array(W_var, dtype='U20'))
    X_s_var = list(np.array(X_s_var, dtype='U20'))  
    A_var = list(np.array(A_var, dtype='U20'))


#print(W_test)
# W = np.concatenate((W_dev, W_test), axis=0)  
# X_s = np.concatenate((X_s_dev, X_s_test), axis=0)
# Y = np.concatenate((Y_dev, Y_test), axis=0) 
# A = np.concatenate((A_dev, A_test), axis=0) 

#Pandas DataFrame
df_Ad = DataFrame(data = A_dev, columns = A_var)
df_Wd = DataFrame(data = W_dev,columns = W_var)
df_xsd = DataFrame(data = X_s_dev,columns = X_s_var)

df_A = DataFrame(data = A_test, columns = A_var)
df_W = DataFrame(data = W_test, columns = W_var)
df_xs = DataFrame(data = X_s_test,columns = X_s_var)

df_Y_dev = DataFrame(data = Y_dev, columns = ['Rul'])
df_Y_test = DataFrame(data = Y_test,columns = ['Rul'])

df_tests = pd.concat([df_A,df_W,df_xs],axis = 1)
del df_tests["P15"]
del df_tests["Fc"]
del df_tests["Ps30"]
print(df_tests)
#print(df_Ad)
# print(df_Wd)
# print(df_xsd)
#print("------------")
#print(df_A)
# print(df_W)
# print(df_xs)
df_dev = pd.concat([df_Wd, df_xsd, df_Ad], axis=1)
#df_dev.drop(['P15','Fc',],axis = 1)
del df_dev["P15"]
del df_dev["Fc"]
del df_dev["Ps30"]

df_Y_dev.drop(df_Y_dev[(df_dev.unit != 16) & (df_dev.unit != 18) & (df_dev.unit != 20)].index, inplace=True)
df_dev.drop(df_dev[(df_dev.unit != 16) & (df_dev.unit != 18) & (df_dev.unit != 20)].index, inplace=True)
print(df_dev.shape)
print(df_Y_dev.shape)

# del df_tests["P15"]
# del df_tests["Fc"]
# del df_tests["Ps30"]
# df_tests.drop(df_tests[(df_tests.unit != 16) & (df_tests.unit != 18) & (df_tests.unit != 20)].index, inplace=True)
# print("pg break")
# print(df_tests)

         unit  cycle   hs      alt      Mach        TRA          T2  \
0        11.0    1.0  1.0  10014.0  0.457506  77.255310  503.176696   
1        11.0    1.0  1.0  10020.0  0.457947  77.255310  503.192949   
2        11.0    1.0  1.0  10029.0  0.458451  77.255310  503.203187   
3        11.0    1.0  1.0  10034.0  0.458136  77.255310  503.158580   
4        11.0    1.0  1.0  10045.0  0.458010  77.255310  503.105629   
...       ...    ...  ...      ...       ...        ...         ...   
1253738  15.0   67.0  0.0  10027.0  0.382473  25.136539  497.040848   
1253739  15.0   67.0  0.0  10022.0  0.382158  25.136539  497.035821   
1253740  15.0   67.0  0.0  10016.0  0.381969  25.136539  497.043961   
1253741  15.0   67.0  0.0  10010.0  0.381717  25.136539  497.047255   
1253742  15.0   67.0  0.0  10003.0  0.381402  25.136539  497.049793   

                T24          T30          T48          T50         P2  \
0        601.369822  1441.086963  1822.407728  1230.069061  11.636834   
1

In [ ]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

train_RNN_x = np.concatenate((df_dev.drop(df_dev[(df_dev.unit != 16)].index).to_numpy(),
                             df_dev.drop(df_dev[(df_dev.unit != 18)].index).to_numpy(),
                             df_dev.drop(df_dev[(df_dev.unit != 20)].index).to_numpy()), axis=0)
                             
train_RNN_y = np.concatenate((df_Y_dev.drop(df_Y_dev[(df_dev.unit != 16)].index).to_numpy(),
                             df_Y_dev.drop(df_Y_dev[(df_dev.unit != 18)].index).to_numpy(),
                             df_Y_dev.drop(df_Y_dev[(df_dev.unit != 20)].index).to_numpy()), axis=0)
#scaling
scaler = MinMaxScaler(feature_range=(0, 1))
train_RNN_x = scaler.fit_transform(train_RNN_x)
train_RNN_y = scaler.fit_transform(train_RNN_y)

train_RNN_y = train_RNN_y.flatten()
#print(train_RNN_x, train_RNN_y, len(train_RNN_x), len(train_RNN_y), train_RNN_y.ndim)

# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(train_RNN_x, (-1, 6, 19))
trainY = numpy.reshape(train_RNN_y, (-1, 6, 1))

Y_test = df_Y_test.to_numpy()
X_test = df_tests.to_numpy()

#print(len(trainX), len(trainY), len(Y_test), len(X_test))

#print("heee")
#print(trainX.ndim, trainY.ndim)

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=20, batch_size=32, verbose=1)
"""
print("heee")

trainY = trainY.flatten()
trainX = trainX.flatten()

print(trainX.ndim, trainY.ndim)

#trainY = scaler.inverse_transform([trainY])
#trainX = scaler.inverse_transform([trainX])

trainX = numpy.reshape(trainX, (-1, 6, 19))
trainY = numpy.reshape(trainY, (-1, 6, 1))
X_test = numpy.reshape(X_test, (-1, 1, 19))
Y_test = numpy.reshape(Y_test, (-1, 1, 89))
print(len(X_test), len(Y_test))

#X_test = scaler.fit_transform(X_test)
#Y_test = scaler.fit_transform(Y_test)

#trainPredict = model.predict(trainX)
#testPredict = model.predict(X_test)

_, acc = model.evaluate(X_test, Y_test,
                            batch_size=32)
print('Accuracy na test skupu:', acc)

# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
Y_test = scaler.inverse_transform([Y_test])

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(Y_test[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
"""

Epoch 1/20
12626/12626 [==============================] - 226s 17ms/step - loss: 0.0012
Epoch 2/20
12626/12626 [==============================] - 218s 17ms/step - loss: 1.0842e-04
Epoch 3/20
12626/12626 [==============================] - 213s 17ms/step - loss: 6.0922e-05
Epoch 4/20
12626/12626 [==============================] - 215s 17ms/step - loss: 4.2662e-05
Epoch 5/20
12626/12626 [==============================] - 215s 17ms/step - loss: 3.4689e-05
Epoch 6/20
12626/12626 [==============================] - 216s 17ms/step - loss: 2.5948e-05
Epoch 7/20
12626/12626 [==============================] - 218s 17ms/step - loss: 2.1405e-05
Epoch 8/20
12626/12626 [==============================] - 222s 18ms/step - loss: 1.9217e-05
Epoch 9/20
12626/12626 [==============================] - 221s 17ms/step - loss: 2.2900e-05
Epoch 10/20
12626/12626 [==============================] - 219s 17ms/step - loss: 1.9489e-05
Epoch 11/20
12626/12626 [==============================] - 220s 17ms/step - loss: 1

'\nprint("heee")\n\ntrainY = trainY.flatten()\ntrainX = trainX.flatten()\n\nprint(trainX.ndim, trainY.ndim)\n\n#trainY = scaler.inverse_transform([trainY])\n#trainX = scaler.inverse_transform([trainX])\n\ntrainX = numpy.reshape(trainX, (-1, 6, 19))\ntrainY = numpy.reshape(trainY, (-1, 6, 1))\nX_test = numpy.reshape(X_test, (-1, 1, 19))\nY_test = numpy.reshape(Y_test, (-1, 1, 89))\nprint(len(X_test), len(Y_test))\n\n#X_test = scaler.fit_transform(X_test)\n#Y_test = scaler.fit_transform(Y_test)\n\n#trainPredict = model.predict(trainX)\n#testPredict = model.predict(X_test)\n\n_, acc = model.evaluate(X_test, Y_test,\n                            batch_size=32)\nprint(\'Accuracy na test skupu:\', acc)\n\n# invert predictions\ntrainPredict = scaler.inverse_transform(trainPredict)\ntrainY = scaler.inverse_transform([trainY])\ntestPredict = scaler.inverse_transform(testPredict)\nY_test = scaler.inverse_transform([Y_test])\n\n# calculate root mean squared error\ntrainScore = math.sqrt(mean_squar

 prosečna kvadratna greška (RMSE) i NASA-ina funkcija bodovanja

In [ ]:
# def delta1(k):
#   return y(k) - y1(k)

# def delta2(k):
#   return math.exp(abs(y(k) - y1(k))*1/13)

# def sum1(mv, k):
#   sum = 0
#   deltak = map(delta1, k)
#   sum = reduce(lambda x, y:x+y, deltak)
#   return sum

# def sum2(mv, k):
#   sum = 0
#   deltak = map(delta2, k)
#   sum = reduce(lambda x, y:x+y, deltak) - 1
#   return sum

# def RMSE(mv, k):
#   return math.sqrt(1/mv * sum1(mv, k))

# def sc(mv, k):
#   return 1/mv * sum2(mv, k)

# score = 0.5 * RMSE(mv, k) + 0.5 * sc(mv, k)
# trainY = trainY.flatten()
# trainX = trainX.flatten()

# print(trainX.ndim, trainY.ndim)

# trainY = scaler.inverse_transform([trainY])
# trainX = scaler.inverse_transform([trainX])

# #trainPredict = model.predict(trainX)
# testPredict = model.predict(X_test)



def delta1(k):
  return (Y_test[k] - testPredict[k]) ** 2

def delta2(k):
  alfa = Y_test(k) - testPredict(k)
  if (alfa > 0):
    return (abs(alfa))*1/10)
  else:
    return (abs(alfa))*1/13)

def sum1(mv):
  sum = 0
  for k in range(mv):
    sum = sum + delta1(k)
  return sum

def sum2(mv,alfa):
  sum = 0
  for k in mv:
    sum = sum + (math.exp(delta2(k))-1)
  return sum

def RMSE(mv):
  return math.sqrt(1/mv*sum1(mv))

def sc(mv, k):
  return 1/mv * sum2(mv, alfa)

alfa = 1/13
mv = len(Y_test)
#gde mv predstavlja broj primera u skupu podataka za validaciju
score = 0.5 * RMSE(mv) + 0.5 * sc(mv, alfa)

1 1


ValueError: ignored